In [ ]:
# importing the Dataset

import pandas as pd
messages = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_train.csv', 
                       usecols=["OriginalTweet", "Sentiment"],encoding='ISO-8859-1')
messages.head()

In [ ]:
messages.shape

In [ ]:
messages['Sentiment'].unique()

In [ ]:
messages.Sentiment.value_counts()

In [ ]:
messages['encoded_cat'] = messages['Sentiment'].astype('category').cat.codes
messages.head()

In [ ]:
messages.Sentiment.value_counts()

In [ ]:
messages.encoded_cat.value_counts()

In [ ]:
messages.info()

In [ ]:
X=list(messages['OriginalTweet'])
X[:5]

In [ ]:
y=list(messages['encoded_cat'])
y[:5]

In [ ]:
round(messages['Sentiment'].value_counts()*100/len(messages),2)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42,stratify=y)
X_train[:5]

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
y_train[:5]

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    logging_steps=100,
    
)

In [ ]:
with training_args.strategy.scope():
    trainer_model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

trainer = TFTrainer(
    model=trainer_model,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
)


trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.predict(test_dataset)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Extremely Negative','Extremely Positive','Negative','Neutral','Positive']
print(classification_report(y_test,
                            trainer.predict(test_dataset)[1],target_names=target_names
                           )
     )